<table style="font-size: 1em; padding: 0; margin: 0;">
<p style="border: 1px solid #ff5733; border-left: 15px solid #ff5733; padding: 10px; text-align:justify;">
    <strong style="color: #ff5733">Deliverable</strong>  
    <br/>The deliverable for Part 2 is a jupyter notebook showing a workflow to create test and training datasets, consisting of folders of imagery and corresponding label imagery, ready for training a semantic segmentation model in keras. This will mostly test your understanding the generic workflow for preparing a dataset to train and test a deep learning model, which is an essential component of the remaining Parts.
    </p>

In [2]:
import os
import urllib.request

## Dataset 1

Text from the Buscomb Live Project:

Use the Sentinel-2 cloudless image service ("option 1") as a means to easily acquire cloudless mosaiced imagery from some of Earth's critical lakes that collectively provide drinking water to countless millions of people. You can use the website to explore and compare changes visually between 2016 and 2018 everywhere, and also 2017 for Europe only. This is the data you will be attempting to test your model (trained on another dataset that is introduced later).

Tip. If you use the download tool to obtain an image tile, you will see some structure in the URL. For example,
https://tiles.maps.eox.at/wms?service=wms&request=getmap&version=1.1.1&layers=s2cloudless-2018&bbox=10.232263646327286,-0.670557747166987,10.334917149745255,-0.5679042437490183&width=4096&height=4096&srs=epsg:4326

Let's break this down a little:

https://tiles.maps.eox.at/wms?service=wms&request=getmap&version=1.1.1: The root of the service that always stays the same. WMS stands for Web Map Service.
layers=s2cloudless2018: The layer to get. Other options are "s2cloudless" for 2016 imagery, and "s2cloudless2017" for 2017 imagery (Europe only).
bbox=10.232263646327286,-0.670557747166987,10.334917149745255,-0.5679042437490183: the bounding box coordinates (region of interest) in the form south Latitude, west Longitude, north Latitude, east Longitude.
width=4096&height=4096: the width and height of the image in number of pixels
srs=epsg:4326: the epsg code (coordinate projection system) that pertains to the bounding box coordinates. In this example the epsg code is 4326, which is GPS coordinates (WGS 84).

Download time-series of images from 17 critically endangered lake systems:
1. <a href="https://en.wikipedia.org/wiki/Lake_Poop%C3%B3">Lake Poopo, Bolivia</a>

2. <a href="https://en.wikipedia.org/wiki/Lake_Urmia">Lake Urmia, Iran</a> 
 
3. <a href="https://en.wikipedia.org/wiki/Lake_Mead">Lake Mead</a>/<a href="https://en.wikipedia.org/wiki/Lake_Mohave">Lake Mojave</a>, USA

4. <a href="https://en.wikipedia.org/wiki/Aral_Sea">Aral sea, Kazahkstan</a>

5. <a href="https://en.wikipedia.org/wiki/Lake_Copais">Lake Copais, Greece</a>

6. <a href="https://en.wikipedia.org/wiki/Ramganga_Dam">Lake Ramganga, India</a>
 
7. <a href="https://en.wikipedia.org/wiki/Qinghai_Lake">Qinghai Lake, China</a> 

8. <a href="https://en.wikipedia.org/wiki/Salton_Sea">Salton Sea, USA</a> 

9. <a href="https://earthobservatory.nasa.gov/images/8991/drying-of-lake-faguibine-mali">Lake Faguibine, Mali</a> 
 
10. <a href="https://en.wikipedia.org/wiki/Mono_Lake">Mono Lake, USA</a> 

11. <a href="https://en.wikipedia.org/wiki/Walker_Lake_(Nevada)">Walker Lake, USA</a>

12. <a href="https://en.wikipedia.org/wiki/Lake_Balaton">Lake Balaton, Hungary</a>

13. <a href="https://en.wikipedia.org/wiki/Lake_Koroneia">Lake Koroneia, Greece</a>

14. <a href="https://en.wikipedia.org/wiki/Lake_Salda">Lake Salda, Turkey</a>

15. <a href="https://en.wikipedia.org/wiki/Lake_Burdur">Lake Burdur, Turkey</a>

16. <a href="https://en.wikipedia.org/wiki/Lake_Mendocino">Lake Mendocino, USA</a>

17. <a href="https://en.wikipedia.org/wiki/Elephant_Butte_Reservoir">Elephant Butte Reservoir, USA</a>

You can read about some of these sites and their problems in a <a href="https://www.nationalgeographic.com/magazine/2018/03/drying-lakes-climate-change-global-warming-drought/">2018 National Geographic article</a>

In [3]:
def download_all_layers(root,layers,bbox,width,height,srs,site):
    #cycle through each layer to download
    for layer in layers:
        # construct a URL string 
        url = root+layer+bbox+width+height+srs
        #download into a filename constructed as "site_layer.jpg"
        urllib.request.urlretrieve(url, site+'_'+layer.split('=')[-1].split('&')[0]+'.jpg')

In [4]:
#Lake Poopo, Bolivia
root = "https://tiles.maps.eox.at/wms?service=wms&request=getmap&version=1.1.1&"
site='poopo'
bbox = 'bbox=-68.66848367000537,-19.687928531849003,-66.67924128546656,-17.8774477409051&'
width = 'width=4096&'
height = 'height=4096&'
layers = ['layers=s2cloudless&', 'layers=s2cloudless-2018&']
srs = 'srs=epsg:4326'

In [5]:
download_all_layers(root,layers,bbox,width,height,srs,site)